# Datacamp - RNA-Seq with Bioconductor in R
RNA-Seq is an exciting next-generation sequencing method used for identifying genes and pathways underlying particular diseases or conditions. As high-throughput sequencing becomes more affordable and accessible to a wider community of researchers, the knowledge to analyze this data is becoming an increasingly valuable skill. Join us in learning about the RNA-Seq workflow and discovering how to identify which genes and biological processes may be important for your condition of interest! We will start the course with a brief overview of the RNA-Seq workflow with an emphasis on differential expression (DE) analysis. Starting with the counts for each gene, the course will cover how to prepare data for DE analysis, assess the quality of the count data, and identify outliers and detect major sources of variation in the data. The DESeq2 R package will be used to model the count data using a negative binomial model and test for differentially expressed genes. Visualization of the results with heatmaps and volcano plots will be performed and the significant differentially expressed genes will be identified and saved.

## Introduction to RNA-Seq theory and workflow
In this chapter we explore what we can do with RNA-Seq data and why it is exciting. We learn about the different steps and considerations involved in an RNA-Seq workflow.

### RNA-Seq Packages
We will be using DESeq2 for performing the differential expression analysis and additional R packages for data wrangling and plotting. Before we can run any analyses, we need to load the following packages `DESeq2`, `RColorBrewer`, `pheatmap`, and `tidyverse`.

In [1]:
# Install library for DESeq2
BiocManager::install("DESeq2")

# Install library for RColorBrewer
BiocManager::install("RColorBrewer")

# Install library for pheatmap
BiocManager::install("pheatmap")


'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.13 (BiocManager 1.30.16), R 4.1.2 (2021-11-01)

Installing package(s) 'DESeq2'

also installing the dependencies 'annotate', 'genefilter', 'locfit', 'geneplotter', 'RcppArmadillo'




package 'annotate' successfully unpacked and MD5 sums checked
package 'genefilter' successfully unpacked and MD5 sums checked
package 'locfit' successfully unpacked and MD5 sums checked
package 'geneplotter' successfully unpacked and MD5 sums checked
package 'RcppArmadillo' successfully unpacked and MD5 sums checked
package 'DESeq2' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\recer\AppData\Local\Temp\Rtmp2L9qwm\downloaded_packages


Old packages: 'backports', 'brio', 'broom', 'cli', 'cpp11', 'crayon', 'DBI',
  'digest', 'dtplyr', 'fansi', 'fs', 'generics', 'glmnet', 'glue', 'jsonlite',
  'knitr', 'languageserver', 'magrittr', 'mvtnorm', 'openssl', 'permute',
  'pillar', 'pkgload', 'raster', 'Rcpp', 'readr', 'remotes', 'repr', 'rex',
  'rJava', 'rjson', 'rlang', 'rsconnect', 'rvest', 'sp', 'stringi', 'testthat',
  'tibble', 'tinytex', 'tzdb', 'uuid', 'vroom', 'withr', 'xfun', 'xgboost',
  'xml2', 'yaml', 'foreign', 'later', 'rpart'

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.13 (BiocManager 1.30.16), R 4.1.2 (2021-11-01)

Warning message:
"package(s) not installed when version(s) same as current; use `force = TRUE` to
  re-install: 'RColorBrewer'"
Old packages: 'backports', 'brio', 'broom', 'cli', 'cpp11', 'crayon', 'DBI',
  'digest', 'dtplyr', 'fansi', 'fs', 'generics', 'g

package 'pheatmap' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\recer\AppData\Local\Temp\Rtmp2L9qwm\downloaded_packages


Old packages: 'backports', 'brio', 'broom', 'cli', 'cpp11', 'crayon', 'DBI',
  'digest', 'dtplyr', 'fansi', 'fs', 'generics', 'glmnet', 'glue', 'jsonlite',
  'knitr', 'languageserver', 'magrittr', 'mvtnorm', 'openssl', 'permute',
  'pillar', 'pkgload', 'raster', 'Rcpp', 'readr', 'remotes', 'repr', 'rex',
  'rJava', 'rjson', 'rlang', 'rsconnect', 'rvest', 'sp', 'stringi', 'testthat',
  'tibble', 'tinytex', 'tzdb', 'uuid', 'vroom', 'withr', 'xfun', 'xgboost',
  'xml2', 'yaml', 'foreign', 'later', 'rpart'



In [3]:
# Load library for DESeq2
library("DESeq2")

# Load library for RColorBrewer
library("RColorBrewer")

# Load library for pheatmap
library("pheatmap")

# Load library for tidyverse
library("tidyverse")